### svp2csv_py

In [1]:
#%% import modules, packages
import pandas as pd
import requests
import os
import glob
import zipfile
from shapely.geometry import Point
import sys
from datetime import datetime
from datetime import timedelta

In [2]:
#%% Functions
def XYZ2SHP_UTM56S(xyz_in_path):
    '''
    input xyz
    output shp file in SHP folder
    '''
    xyz_dirname = os.path.dirname(xyz_in_path)
    os.chdir(xyz_dirname)
    
    #31: add headers to the file 'X' 'Y' 'Z'
    infile = open(xyz_in_path)
    inLines = infile.read()
    infile.close()
    outLines = 'X Y Z\n'+inLines
    outLines = outLines.replace(' ',",")
    outfile_location = xyz_in_path[:-4]+'_ENZ.csv'
    outfile = open(outfile_location,'w')
    outfile.write(outLines)
    outfile.close()
    
    #Create geopandas dataframe
    inXYZdf = pd.read_csv(outfile_location)
    geometry = [Point(lonlat) for lonlat in zip(inXYZdf['X'],inXYZdf['Y'])]
    crs = {'init': 'epsg:28350'}
    geopdf = gpd.GeoDataFrame(inXYZdf,crs=crs,geometry=geometry)
    
    # Plot the map
    geopdf.plot(marker='o', figsize = (6,6),color='g', markersize=5);

    #70 Create a new export folder
    try:
        os.mkdir('SHP')
        os.mkdir('csv')
    except:
        pass
    #71: write to SHP file
    try:
        geopdf.to_file(os.path.join(xyz_dirname,'SHP',os.path.basename(outfile_location)[:-4]+'.shp'))
    except:
        pass

#################################################################################
#Define a function to list all Geotiff files in a folder
def nameList_F_withExt(InputFolder,filterString="*"):
    '''
    pathList_F_ext(InputFolder,filterString="*")
    list all files and folders in InputFolder
    return a list of names for every file and folder matching folderString
    file includes extention (ext) information
    '''
    os.chdir(InputFolder) #change working folder
    return glob.glob(filterString)

def pathList_F_ext(InputFolder,filterString="*"):
    """
    pathList_F_ext(InputFolder,filterString="*")
    list all files and folders in InputFolder
    return a list of paths for every file and folder matching folderString
    file includes extention (ext) information
    """
    import glob
    os.chdir(InputFolder) #change working folder
    baseName_FolderList = glob.glob(filterString) #list all the folder list in InputFolder
    pathList = []
    for folder in baseName_FolderList:
        pathList.append(os.path.abspath(folder))
    return pathList

###############################################################################
def newFolderF(containFolder,newFolder='newFolder'):
    '''
    create new folder
    containFolder: the location of new created foler
    newFolder: new folder name
    Step 1: change dir to containFolder
    Step 2: check if the new folder is existed or not
    Step 3: if not make new folder using os.mkdir
    return the function result to new path.
    '''
    os.chdir(containFolder)
    if os.path.exists(containFolder+"\\"+newFolder): #if newFolder exists
        next
        #print ("Already exists:",containFolder+"\\"+newFolder)
    else:
        os.mkdir(newFolder)
    return containFolder+"\\"+newFolder #return the abspath
    
#From standard Date to Julian date
def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday
    return(jdate)
datestdtojd('2018-11-01')
#From Julian date to standard Date
def jdtodatestd (jdate):
    fmt = '%Y-%j'
    datestd = datetime.strptime(jdate, fmt).date()
    return(datestd)
jdtodatestd('2018-305')

#
def dms_to_dd(d, m, s):
    if d[0]=='-':
        dd = float(d) - float(m)/60 - float(s)/3600
    else:
        dd = float(d) + float(m)/60 + float(s)/3600
    return dd


## Caris format svp to csv files list table

In [43]:
path2file = "11_Deliverables/03_Bathymetry/05_SoundVelocityProfiles/SVP_CARIS_Format/svp/"+svp

### CSV2SHP_Arcpy

In [54]:
# CARIS format
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_CARIS_Format"
in_table = r"SI1001_SVP_CARIS_Format.csv"
out_feature_class = "SI1001_SVP_CARIS_Format.shp"
x_coords = "lon_deg"
y_coords = "lat_deg"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(4326) #CRS: WGS84 4326
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
# total rows
print(arcpy.GetCount_management(out_feature_class))

#note location is very bad, do not use svp from files

775


# RAW data

In [55]:
# XYTableToPoint.py
#CRS: WGS84 4326
# import system modules 
import arcpy
# Set environment settings
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
# Set the local variables
in_table = r"SI1001_SVP_SSDM.csv"
out_feature_class = "SI1001_SVP_WGS84.shp"
x_coords = "Lon_E"
y_coords = "Lat_S"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(4326)
# Make the XY event layer...
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
# Print the total rows
print(arcpy.GetCount_management(out_feature_class))

#not good

102


In [94]:
# WGS 84 / UTM zone 56S 32756
# Set environment settings
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
# Set the local variables
in_table = r"SI1001_SVP_RAW.csv"
out_feature_class = "SI1001_SVP_UTM56S.shp"
x_coords = "E_UTM56S"
y_coords = "N_UTM56S"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(32756)
# Make the XY event layer...
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
# Print the total rows
print(arcpy.GetCount_management(out_feature_class))

#very good

0


In [4]:
# Set environment settings
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
 
# Set local variables
inTable = "SI1001_SVP_UTM56S.dbf"
outLocation = arcpy.env.workspace
outTable = "SI1001_SVP_RAW_Edit1.csv"

# Execute TableToTable
arcpy.TableToTable_conversion(inTable, outLocation, outTable)

<Result 'E:\\HIPP\\11_Deliverables\\03_Bathymetry\\05_SoundVelocityProfiles\\SVP_Raw\\SI1001_SVP_RAW_Edit1.csv'>

In [5]:
# WGS 84 / UTM zone 56S 32756
# Set environment settings
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
# Set the local variables
in_table = r"SI1001_SVP_RAW_Edit1.csv"
out_feature_class = "SI1001_SVP_UTM56S_R2.shp"
x_coords = "E_UTM56S"
y_coords = "N_UTM56S"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(32756)
# Make the XY event layer...
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
# Print the total rows
print(arcpy.GetCount_management(out_feature_class))

#very good

102
